In [1]:
# Author: Mishma Mariyam Raju
# Created on July 12 10:15 am

In [2]:
import struct
import numpy as np
import math
from scipy.optimize import fsolve
import random
import os

In [3]:
def solve_for_G(mu):
    func = lambda G : G - mu * np.exp(G - mu)
    
    '''
    G = np.linspace(0.000, 1.000, 1000)
    plt.plot(G, func(G))
    plt.xlabel("G")
    plt.ylabel("expression value")
    plt.grid()
    plt.show()
    '''

    G_initial_guess = 0.5
    G_soln = fsolve(func, G_initial_guess)
    #print("The solution is G = %f" % G_soln)
    #print("at which the value of the expression is %f" % func(G_soln))
    return G_soln[0]

In [4]:
def ln_stirling_2nd(n, k):
    mu = n/k
    G = solve_for_G(mu)
    #G = 0.5
    return 0.5 * (math.log(mu - 1) - math.log(mu * (1 - G))) + \
        (n - k) * (math.log(mu - 1) - math.log(mu - G)) + \
            n * math.log(k) - k * math.log(n) + k * (1 - G) + \
                ln_n_fact(n) - ln_n_fact(n - k) - ln_n_fact(k)

In [5]:
def ln_n_fact(n):
    if(n > 8):
        return 0.5 * math.log(2*math.pi) - n + (n + 0.5) * math.log(n)
    else: 
        return math.log(math.factorial(n))

In [6]:
def P(x, n, l):
    if(x == int(l/n) or x == 1):
        return math.e ** (ln_n_fact(2 ** n) - ln_n_fact(2**n - x) + ((l%n) - l) * math.log(2))
    else:
        Term_1 = ln_n_fact(2 ** n) - ln_n_fact(2**n - x) 
        #Term_2 = ln_n_fact(int(l/n)) - ln_n_fact(int(l/n) - x) + x * math.log(int(l/n))
        Term_2 = ln_stirling_2nd(int(l/n), x)
        #Term_2 = math.log(stirling(int(l/n), x))
        Term_3 = ((l%n) - l) * math.log(2)
        #print(Term_1, Term_2, Term_3)
        #print(math.e ** (Term_1 + Term_2 + Term_3))
        return math.e ** (Term_1 + Term_2 + Term_3)

In [7]:
def E(n, l):
    E = 0.0
    sump = 0.0
    p = 0.0
    for x in range(1, min(2 ** n + 1, int(l/n)+1)):
        p = P(x, n, l)
        sump += p
        E += p * x
    '''
    p = [P(x, n, l) for x in range(1, min(2 ** n + 1, int(l/n)+1))]
    #print(p)
    E_of_n_l = [p[x-1] * x for x in range(1, min(2 ** n + 1, int(l/n)+1))]
    print("E(", n, "\b,", l, "\b)", int(sum(E_of_n_l)))
    '''
    #print(sump)
    print("E(", n, "\b,", l, "\b) = ", E)
    return E

In [8]:
def find_length_of_bit_file(bit_file_name):
    size_in_bytes = os.path.getsize(bit_file_name)
    size_of_one_character = 1.0
    return int(size_in_bytes/size_of_one_character)

In [9]:
def find_dp(bit_file_name, n, offset = 0):
    corpus = open(bit_file_name)
    file = open("distinct_patterns.txt", "w")
    for i in range(offset, n + offset):
        file.write(corpus.read(1))
    file.write("\n")
    file.close()
    
    length_of_bit_file = find_length_of_bit_file(bit_file_name)
    no_of_distinct_patterns = 1
    
    for i in range(offset + n, length_of_bit_file - n, n):
        flag = 1
        pattern = []
        for j in range(0, n):
            pattern.append(int(corpus.read(1)))
        #print(pattern)
        with open('distinct_patterns.txt') as f:
            for index, line in enumerate(f):
                #print("Line {}: {}".format(index, line.strip()))
                j = []
                for ch in line.strip():
                    j.append(int(ch))
                    pass
                if(np.array_equal(pattern, j)):
                    flag = 0
        file = open("distinct_patterns.txt", "a")  
        if(flag == 1):
            for num in pattern:
                file.write(str(num))
            file.write("\n")
            no_of_distinct_patterns += 1
        file.close()
    print(no_of_distinct_patterns, end = ", ")
    return no_of_distinct_patterns

In [10]:
def CS(bit_file_name, cs_file, offset = 0):
    file = open(cs_file, 'w')
    for n in range(1, 45):
        file.write(str(1 - find_dp(bit_file_name, n, offset)/E(n, find_length_of_bit_file(bit_file_name))) + ",")  
    file.close()
    return

In [11]:
cs_file = 'CSs/cs_bit_ASCII_128_all.txt'
bit_file_name = 'Corpuses/bit_ASCII_128_all.txt'

CS(bit_file_name, cs_file)

2, E( 1, 8953) =  1.999999999998181
4, E( 2, 8953) =  4.000000000032742
8, E( 3, 8953) =  7.999999999949068
16, E( 4, 8953) =  15.99999999992724
32, E( 5, 8953) =  32.00000000014552
64, E( 6, 8953) =  64.00000000031507
128, E( 7, 8953) =  127.99478931720962
209, E( 8, 8953) =  252.81892314367454
285, E( 9, 8953) =  439.2348675486044
373, E( 10, 8953) =  597.1824740944869
406, E( 11, 8953) =  671.5928626520324
416, E( 12, 8953) =  683.003799588665
434, E( 13, 8953) =  661.9804284483304
224, E( 14, 8953) =  630.3676283392947
414, E( 15, 8953) =  591.0915695537193
410, E( 16, 8953) =  556.6312741425173
387, E( 17, 8953) =  524.948315155376
365, E( 18, 8953) =  496.5302692290477
357, E( 19, 8953) =  470.78902315211036
348, E( 20, 8953) =  446.90498587661085
263, E( 21, 8953) =  425.9568550796303
312, E( 22, 8953) =  405.9804045638181
307, E( 23, 8953) =  388.99101306876736
293, E( 24, 8953) =  372.9958572421843
281, E( 25, 8953) =  357.99813071451007
271, E( 26, 8953) =  343.9990972595297
